In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

# Define transformations for the training and test sets
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the CIFAR-10 dataset
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)


In [5]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(128 * 4 * 4, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleCNN(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [6]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

    # Validation loop
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Loss: {test_loss/len(test_loader):.4f}, Accuracy: {100 * correct / total:.2f}%')


Epoch [1/20], Loss: 1.5348
Test Loss: 1.1718, Accuracy: 57.59%
Epoch [2/20], Loss: 1.1670
Test Loss: 0.9868, Accuracy: 64.93%
Epoch [3/20], Loss: 1.0070
Test Loss: 0.8943, Accuracy: 68.65%
Epoch [4/20], Loss: 0.9070
Test Loss: 0.8470, Accuracy: 70.42%
Epoch [5/20], Loss: 0.8405
Test Loss: 0.7951, Accuracy: 71.52%
Epoch [6/20], Loss: 0.7868
Test Loss: 0.7370, Accuracy: 74.06%
Epoch [7/20], Loss: 0.7403
Test Loss: 0.6825, Accuracy: 76.24%
Epoch [8/20], Loss: 0.7066
Test Loss: 0.6716, Accuracy: 76.78%
Epoch [9/20], Loss: 0.6733
Test Loss: 0.6649, Accuracy: 77.10%
Epoch [10/20], Loss: 0.6400
Test Loss: 0.6552, Accuracy: 77.56%
Epoch [11/20], Loss: 0.6200
Test Loss: 0.6307, Accuracy: 78.18%
Epoch [12/20], Loss: 0.6009
Test Loss: 0.6444, Accuracy: 77.70%
Epoch [13/20], Loss: 0.5794
Test Loss: 0.6163, Accuracy: 78.86%
Epoch [14/20], Loss: 0.5636
Test Loss: 0.6152, Accuracy: 78.83%
Epoch [15/20], Loss: 0.5491
Test Loss: 0.6170, Accuracy: 79.03%
Epoch [16/20], Loss: 0.5286
Test Loss: 0.6029, Ac